In [351]:
import copy
import math
import random

class Field:
    
    def __init__(self, width, height, critical=4):
        self.width = width
        self.height = height
        self.critical = critical
        
        assert(self.width >= self.critical)
        assert(self.height >= self.critical)
        
        self.data = [' ' for i in range((self.height) * self.width)]
        self.frozen_data = self.data[:]


    def set(self, line, column, val):
        num = line * self.width + column
        assert(num < len(self.data))
        
        self.data[num] = val if val in ['x', 'o'] else None


    def get(self, line, column):
        num = line * self.width + column
        if num < len(self.data):
            return self.data[num]
        
        return None


    def print(self):
        for l in range(self.height):
            for c in range(self.width):
                end = "\n" if c == self.width - 1 else ''
                val = self.data[l * self.width + c]
                
                if val != self.frozen_data[l * self.width + c]:
                    val = val.upper()
                
                if val is None:
                    val = ' '
                print(val, end=end)

    
    def count_error(self): 
        return len(self.get_error_cells())

    
    def get_error_cells(self):
        res = []
        for coords in self.generate_position():
            vals = map(lambda coord: self.get(coord[0], coord[1]), coords)
            if len(set(vals)) == 1:
                res.append(coords)
                
        return res
        
    
    def get_field_no_errors(self):
        field = copy.deepcopy(self)
        for coords in self.get_error_cells():
            for coord in coords:
                for l in range(coord[0] - self.critical + 1, coord[0] + self.critical - 1):
                    if l < 0 or l >= self.height:
                        continue
                    
                    for c in range(coord[1] - self.critical + 1, coord[1] + self.critical - 1):
                        if c < 0 or c >= self.width:
                            continue
                        if self.frozen_data[l * self.width + c] is None:
                            field.set(l, c, ' ')
        
        field.freeze()
        return field
        
        
    def get_empty_cells(self):
        for l in range(0, self.height):
            for c in range(0, self.width):
                if self.get(l, c) is None:
                    yield (l, c)
        
    
    def freeze(self):
        self.frozen_data = self.data[:]
    
    
    def reset_frozen(self):
        self.data = self.frozen_data[:]
    
    
    def generate_position(self):
        # vertical
        for l in range(0, self.height - (self.critical - 1)):
            for c in range(0, self.width):
                 yield self.count_position_vertical(l, c)
                
        # horizontal
        for l in range(0, self.height):
            for c in range(0, self.width - (self.critical - 1)):
                yield self.count_position_horizontal(l, c)
                
        # diagonal
        for l in range(0, self.height - (self.critical - 1)):
            for c in range(0, self.width - (self.critical - 1)):
                yield self.count_position_diagonal_td(l, c)
                yield self.count_position_diagonal_dt(l, c)
        
                
    def count_position_vertical(self, l, c):
        return [(l + i, c) for i in range(0, self.critical)]
    
    
    def count_position_horizontal(self, l, c):
        return [(l, c + i) for i in range(0, self.critical)]
    
    
    def count_position_diagonal_td(self, l, c):
        return [(l + i, c + i) for i in range(0, self.critical)]

    
    def count_position_diagonal_dt(self, l, c):
        return [(l - i + self.critical, c + i) for i in range(0, self.critical)]
                

class BooleanGenetic:
    
    def __init__(self, field, initial=None, best=None):
        self.field = field
        self.initial = initial or int(field.width * field.height * 2)
        self.best = best or int(math.sqrt(self.initial)) + 5
        
        self.empty_map = list(self.field.get_empty_cells())


    def generate_initial(self):
        children = []
        
        for i in range(self.initial):
            child = [random.randint(0, 1) for empty in self.empty_map]
            children.append(child)
        
        return children


    def estimate_child(self, child):
        self.fill_field(child)
        return self.field.count_error()
        
    
    def fill_field(self, child):
        self.field.reset_frozen()
        for k in range(len(self.empty_map)):
            cell = self.empty_map[k]
            gen = child[k]
            self.field.set(cell[0], cell[1], 'x' if gen else 'o')
        return self.field


    def estimate_children(self, children):
        children_data = [(child, self.estimate_child(child)) for child in children]
        children_data = sorted(children_data, key=lambda c: c[1])
        return children_data
    
    
    def merge(self, mom, dad):
        return [mom[x] if random.randint(0, 1) > 0 else dad[x] for x in range(len(self.empty_map))]
        
    
    def mutate(self, child):
        return [gen if random.randint(0,100) < 95 else (gen + 1) % 2 for gen in child]
        
    
    def merge_best(self, estimated_children):
        best = estimated_children[:self.best]
        
        new_children = []
        for estimated_mom in best:
            for estimated_dad in best:
                new_children.append(self.merge(estimated_mom[0], estimated_dad[0]))
    
        return new_children
    
    
    def mutate_children(self, children):
        return list(map(self.mutate, children))
        
        
def read_field(file):
    with open(file, 'r') as f:
        size = f.readline()
        (width, height) = size.split()
        
        width = int(width)
        height = int(height)
        
        field = Field(width, height)
        
        for l in range(height):
            line = f.readline()
            for j in range(width):
                field.set(l, j, line[j])

        field.freeze()
        return field

        
def genetic(field, initial=100, best=10, generations=100):
    g = BooleanGenetic(field, initial=initial, best=best)
    children = g.generate_initial()

    end = False
    
    for gen in range(1, generations + 1):
        if end:
            break
            
        children = g.mutate_children(children)
        estimates = g.estimate_children(children)
        children = g.merge_best(estimates)

        for i in estimates:
            if i[1] == 0:
                end = True
                break
    
    return (g, estimates, children)
    

def exec_genetic(field): 
    (g, estimates, children) = genetic(field)
    for e in estimates:
        print(e[0], e[1])
    best_one = estimates[0]
    print('score: {}'.format(best_one[1]))
    field = g.fill_field(best_one[0])
    field.print()
    
    return (best_one, g, estimates, children)
    

field = read_field('field1.txt')
field.print()
print('')
(best_one, g, estimates, children) = exec_genetic(field)

i = 0
while i < 5:
    if best_one[1] != 0:
        print('')
        print('Trying to improve result...')
        (best_one, g, estimates, children) = exec_genetic(field.get_field_no_errors())
    i += 1

x x xoxo
 oxx oxo
oo o  ox
o o o ox
x x o ox

[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 0
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 0
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 0
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 0
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 0
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 1
[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1] 1
[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1] 1
[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1] 1
[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 1
[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1] 1
[0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1] 1
[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1,